# Objetivo:

- Recuperar os dados de Empresas e Sócios do site da Receita Federal de forma dinâmica;
- Selecionar Dinâmicamente o diretório mais recente;
- Escolher qual o arquivo que quer baixar ou baixar todos os arquivos;
- Salvar o .zip na Camada Bronze;
- Extrair o arquivo de dentro do .zip

### Import das Bibliotecas

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile

### Definição do Diretório Bronze

In [2]:
# Diretório local onde salvar os arquivos
BRONZE_DIR = "../data/bronze"
os.makedirs(BRONZE_DIR, exist_ok=True)

### Site da Receita Federal

#### Acessar o Site

In [22]:
# URL base
BASE_URL = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"

In [23]:
# Buscar página inicial
resp = requests.get(BASE_URL)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

#### Navegar no Site

In [24]:
# Pegar todos os diretórios
dirs = [a["href"] for a in soup.find_all("a", href=True) if a["href"].endswith("/") and a["href"][0].isdigit()]

In [25]:
# Ordenar diretórios por nome (o mais recente é o último)
dirs.sort()
latest_dir = dirs[-1]
print(f"Diretório mais recente encontrado: {latest_dir}")

Diretório mais recente encontrado: 2025-08/


In [29]:
# Acessar a pasta mais recente
resp_latest = requests.get(BASE_URL + latest_dir)
resp_latest.raise_for_status()
soup_latest = BeautifulSoup(resp_latest.text, "html.parser")

In [32]:
# Procurar arquivos Empresas.zip e Socios.zip
links = [a["href"] for a in soup_latest.find_all("a", href=True)]

# Recupera todos os arquivos de Empresas e Socios
empresas_files = [l for l in links if "Empresas" in l and l.endswith(".zip")]
socios_files = [l for l in links if "Socios" in l and l.endswith(".zip")]

# Pegar o primeiro arquivo (ou o último, dependendo da sua preferência)
empresas_file = empresas_files[0] # ou empresas_files[-1] para o último
socios_file = socios_files[0]  # ou socios_files[-1]

print(f"Arquivos Empresas encontrados: {empresas_files}")
print(f"Arquivo Empresas selecionado: {empresas_file}")
print(f"Arquivos Socios encontrados: {socios_files}")
print(f"Arquivo Socios selecionado: {socios_file}")

Arquivos Empresas encontrados: ['Empresas0.zip', 'Empresas1.zip', 'Empresas2.zip', 'Empresas3.zip', 'Empresas4.zip', 'Empresas5.zip', 'Empresas6.zip', 'Empresas7.zip', 'Empresas8.zip', 'Empresas9.zip']
Arquivo Empresas selecionado: Empresas0.zip
Arquivos Socios encontrados: ['Socios0.zip', 'Socios1.zip', 'Socios2.zip', 'Socios3.zip', 'Socios4.zip', 'Socios5.zip', 'Socios6.zip', 'Socios7.zip', 'Socios8.zip', 'Socios9.zip']
Arquivo Socios selecionado: Socios0.zip


#### Baixar e Extrair Arquivos

In [33]:
# Função para baixar arquivo
def download_file(file_url, folder):
    filename = file_url.split("/")[-1]
    filepath = os.path.join(folder, filename)
    print(f"Baixando {filename}...")
    r = requests.get(BASE_URL + latest_dir + file_url, stream=True)
    r.raise_for_status()
    with open(filepath, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Salvo em {filepath}")
    return filepath

In [34]:
# Função para extrair .zip
def extract_zip(zip_path, extract_to):
    print(f"Extraindo {zip_path}...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Arquivos extraídos em {extract_to}")

In [35]:
# Baixar e descompactar os arquivos
for file in [empresas_file, socios_file]: # Caso queira baixar todos os arquivos trocar por empresas_files e socios_files
    zip_path = download_file(file, BRONZE_DIR)
    extract_zip(zip_path, BRONZE_DIR)

Baixando Empresas0.zip...
Salvo em ../data/bronze/Empresas0.zip
Extraindo ../data/bronze/Empresas0.zip...
Arquivos extraídos em ../data/bronze
Baixando Socios0.zip...
Salvo em ../data/bronze/Socios0.zip
Extraindo ../data/bronze/Socios0.zip...
Arquivos extraídos em ../data/bronze
